In [29]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import KBinsDiscretizer
import math
import warnings

In [9]:
df = pd.read_csv ('brca_metabric/data_expression_median.csv')# will import datset


In [10]:
df


,Hugo_Symbol,MB-0362,MB-0346,MB-0386,MB-0574,MB-0503,MB-0641,MB-0201,MB-0218,MB-0316,...,MB-6122,MB-6192,MB-4820,MB-5527,MB-5167,MB-5465,MB-5453,MB-5471,MB-5127,MB-4313
0,RERE,8.676978,9.653589,9.033589,8.814855,9.274265,9.286585,8.437347,8.569973,8.892102,...,8.756024,8.804947,9.991215,9.595923,9.637249,8.131637,9.606915,9.049296,8.858622,8.415867
1,RNF165,6.075331,6.687887,5.910885,5.628740,5.908698,6.206729,6.095592,6.383530,5.773691,...,7.101087,5.601472,7.103160,6.418987,6.203370,9.101942,7.427494,6.850000,6.550450,6.831722
2,CD049690,5.453928,5.454185,5.501577,5.471941,5.531743,5.372668,5.693519,5.401276,5.567946,...,5.344674,5.364348,5.504866,5.277314,5.439207,5.423027,5.534115,5.339346,5.566071,5.541395
3,BC033982,4.994525,5.346010,5.247467,5.316523,5.244094,5.167365,5.189106,5.416517,5.339133,...,5.300466,4.861101,4.980904,5.436855,5.346776,4.939292,5.062191,5.166765,5.140141,5.266802
4,PHF7,5.838270,5.600876,6.030718,5.849428,5.964661,5.783374,5.737572,5.923928,5.902223,...,5.653481,5.922894,6.181493,5.992153,6.230316,5.644587,5.927409,6.117095,5.936371,7.408960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24363,CC2D1A,6.353215,6.132355,6.366335,6.424048,6.214301,6.175542,6.003532,6.501156,6.617991,...,6.111693,6.180190,6.026150,6.193474,6.437007,6.269748,7.088676,6.110477,6.316304,7.594277
24364,CB986545,4.836483,5.316819,5.466419,5.193150,5.274600,5.299248,5.555575,5.241082,5.220612,...,5.304611,4.970910,5.326566,5.234638,5.330035,5.337776,5.216496,5.427567,4.977467,5.690294
24365,IGSF9,7.304643,7.933324,7.580336,6.903654,6.839417,6.326814,8.375567,6.534520,7.143399,...,6.668100,6.580473,5.927223,6.166514,7.342794,7.288315,7.248336,7.596215,6.620605,7.437116
24366,DA110839,5.251843,5.450611,5.235394,5.091927,5.315224,5.543680,5.444825,5.420087,5.307056,...,5.583404,5.222982,5.487438,5.485667,5.369609,5.359223,5.544276,5.405179,5.631662,5.590516


In [11]:
df_t=df

In [12]:
df_t=df_t.set_index('Hugo_Symbol')

In [13]:
df_t=df_t.reset_index()

In [14]:
df_t=df_t.T

In [15]:
df_t=df_t.reset_index()

In [16]:
df_t.iloc[0:,9:10]

,8
0,SLC17A3
1,5.70238
2,5.52179
3,5.68953
4,5.43913
...,...
1900,5.59773
1901,5.55055
1902,5.49755
1903,5.6903


In [17]:
df_t.iloc[1:,0]

1       MB-0362
2       MB-0346
3       MB-0386
4       MB-0574
5       MB-0503
         ...   
1900    MB-5465
1901    MB-5453
1902    MB-5471
1903    MB-5127
1904    MB-4313
Name: index, Length: 1904, dtype: object

In [18]:
genes=np.array(df_t.iloc[0,1:])
genes

array(['RERE', 'RNF165', 'CD049690', ..., 'IGSF9', 'DA110839', 'FAM71A'],
      dtype=object)

In [19]:
patients=np.array(df_t.iloc[1:,0])
patients

array(['MB-0362', 'MB-0346', 'MB-0386', ..., 'MB-5471', 'MB-5127',
       'MB-4313'], dtype=object)

In [20]:
ds=df_t.iloc[1:,1:]

In [21]:
ds=ds.fillna(0)

In [22]:
ds=ds.T

In [23]:
X=np.array(ds)

In [160]:
X

array([[8.67697843, 9.65358947, 9.03358867, ..., 9.04929646, 8.85862168,
        8.41586735],
       [6.07533084, 6.68788707, 5.91088464, ..., 6.85000036, 6.55045039,
        6.83172229],
       [5.45392767, 5.45418498, 5.50157743, ..., 5.33934575, 5.56607125,
        5.54139538],
       ...,
       [7.30464265, 7.93332437, 7.58033627, ..., 7.59621495, 6.62060542,
        7.43711579],
       [5.25184266, 5.45061128, 5.2353943 , ..., 5.40517883, 5.63166168,
        5.59051551],
       [5.04959138, 5.31679043, 5.46161666, ..., 5.09433903, 5.35070827,
        5.3292088 ]])

In [25]:
enc = KBinsDiscretizer(n_bins=3, encode='ordinal',strategy='kmeans')


In [155]:
X[0:6].shape

(6, 1904)

In [162]:
X


(24368, 1904)

In [169]:
X_binned=valuess

In [32]:
warnings.filterwarnings("ignore", category=UserWarning)

In [186]:
X_binned=[]


In [187]:
initialise_kbinned=(enc.fit_transform(X[0:3]))

enc.bin_edges_[0]

array([5.45392767, 7.22080384, 8.67697843])

In [188]:
X_binned=initialise_kbinned
X_binned

array([[1., 2., 1., ..., 2., 2., 2.],
       [0., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [189]:
for x in range(3,24368,3):
    valuess=(enc.fit_transform(X[x:x+3]))
    print(x,x+3,enc.bin_edges_[0])
    X_binned=np.concatenate((X_binned, valuess))
        

3 6 [4.99452496 5.41639755 6.11788668 6.39750322]
6 9 [5.25946052 5.48091963 6.8042977  7.90621666]
9 12 [5.27567579 5.30426922 6.13180197 6.93074128]
12 15 [5.33203945 5.36193219 5.41786026 5.4438956 ]
15 18 [5.50245922 5.80936388 6.38558868 6.65490883]
18 21 [ 6.1708484  8.1057716 10.0142622]
21 24 [5.28251665 5.33817003 5.42382653 5.45382965]
24 27 [5.1802515  6.1614242  7.50407942]
27 30 [ 6.33894201  7.59596416  9.50978803 10.16658975]
30 33 [5.54156286 6.66849414 7.8309403 ]
33 36 [5.19057539 5.26266604 6.81412466 8.29349263]
36 39 [5.5574412  5.67627349 6.96441412 8.13372245]
39 42 [5.30731905 5.60797253 6.18617849 6.46373099]
42 45 [5.42002212 5.45442823 5.83555056 6.18226679]
45 48 [5.11338953 5.22830355 5.38984461 5.43647166]
48 51 [5.50774127 6.05886161 7.13258055 7.65517914]
51 54 [5.758287   6.720126   8.799996   9.91802701]
54 57 [ 5.86593494  8.39935523 10.88794245]
57 60 [5.49533938 6.32847955 7.88429714 8.60697456]
60 63 [5.30053736 5.42660864 6.04992091 6.5471619 ]
63

495 498 [5.41513134 5.42291136 5.54994773 5.66920408]
498 501 [ 5.45801262  5.48276329  8.27503688 11.0425598 ]
501 504 [5.26292074 7.41049458 9.44206094]
504 507 [5.59265663 6.94648257 8.12224063]
507 510 [5.23627884 6.53592749 7.78589305]
510 513 [5.36338447 5.46301728 5.61830021 5.67395034]
513 516 [ 6.65228991  7.8638013   9.71701297 10.35871325]
516 519 [5.23371485 5.2602273  6.04698863 6.80723752]
519 522 [6.72417099 6.94043047 7.29091699 7.42514403]
522 525 [5.62076961 6.0193559  6.34378232]
525 528 [ 5.42740003  7.14683228  9.79143148 10.71659844]
528 531 [5.11573544 6.03675568 7.20834347 7.45891103]
531 534 [5.22443942 5.27298936 5.82682773 6.33211615]
534 537 [5.32210378 5.42316018 5.49756504]
537 540 [5.18330956 5.7108198  6.5518581  6.86538616]
540 543 [5.29430506 5.37274587 7.04876825 8.64634982]
543 546 [6.21260516 6.56124729 6.90101861]
546 549 [5.54306008 6.10012187 7.32363294 7.99008221]
549 552 [5.73117353 5.74590073 7.59231818 9.42400843]
552 555 [5.17902643 5.360074

990 993 [5.26958931 5.35588201 5.47555712 5.50893954]
993 996 [5.32424793 5.44699166 5.62795727 5.68617915]
996 999 [6.353118   7.06753872 8.22220931 8.66245917]
999 1002 [5.40569497 5.55028068 5.7842805  5.87369461]
1002 1005 [5.49217521 7.07563541 8.8890742  9.11905277]
1005 1008 [ 5.43740335  5.53135928  9.25295648 12.88059775]
1008 1011 [5.17486102 5.36108279 5.90240106 6.25749757]
1011 1014 [5.43932374 5.82882207 7.06252436 7.90672832]
1014 1017 [5.24331362 5.6410505  5.97299722]
1017 1020 [5.65748803 6.05295881 7.61023069 8.7720318 ]
1020 1023 [5.44533371 5.78354402 7.14113334 8.16051234]
1023 1026 [5.14055786 5.86125085 6.45083416]
1026 1029 [5.61622523 6.33929123 7.09016266 7.1179681 ]
1029 1032 [5.42071922 6.20805214 6.95834894]
1032 1035 [5.58248993 5.61615979 6.74845692 7.8470842 ]
1035 1038 [5.43889471 5.57229195 5.83664508 5.96760098]
1038 1041 [ 5.52035893  5.81017228  8.09147912 10.08297261]
1041 1044 [5.40701604 5.62520522 5.91735842 5.99132244]
1044 1047 [5.17572033 5.

1461 1464 [5.21098431 5.49395435 6.78657474 7.79622509]
1464 1467 [6.29164973 6.58625341 8.26209452 9.64333195]
1467 1470 [5.33951502 5.70061052 6.30378791 6.54586979]
1470 1473 [ 5.48964296  7.0189504   9.68006448 10.81187113]
1473 1476 [5.16046732 5.20961834 5.31481596 5.37086258]
1476 1479 [ 5.58029948  5.90265812  8.42404233 10.62306791]
1479 1482 [5.3028533  5.39835338 5.49582871 5.49780397]
1482 1485 [5.62159586 6.28573106 6.867712  ]
1485 1488 [5.2780234  5.48209828 5.73009274]
1488 1491 [5.26179522 6.69248729 8.06187133]
1491 1494 [5.4167453  5.56278656 5.8441242  5.97942057]
1494 1497 [5.85047455 6.16484726 6.93000935 7.38079875]
1497 1500 [5.28918146 6.55829705 8.15694689 8.48648116]
1500 1503 [5.21629568 6.86985544 8.4715764 ]
1503 1506 [6.36186433 6.43854415 6.78945752 7.06369106]
1506 1509 [ 5.4411535   5.7566474   8.08429832 10.09645533]
1509 1512 [5.18155351 5.31972124 5.51584097 5.57379297]
1512 1515 [5.22073796 5.30255496 5.42700895 5.46964595]
1515 1518 [5.2869549  5.

1935 1938 [5.66068736 5.91362835 7.56633682 8.96610431]
1938 1941 [5.36618107 6.55564789 7.96658805 8.1880614 ]
1941 1944 [5.38901157 5.43743809 5.70600308 5.92614154]
1944 1947 [ 5.30013501  7.97287419 10.4986979 ]
1947 1950 [5.28324084 6.10810813 7.08802185]
1950 1953 [5.3352516  5.33794286 5.89779674 6.45495937]
1953 1956 [5.4069043  5.56277966 6.44634286 7.1740307 ]
1956 1959 [5.49402275 5.5210334  5.6016752  5.65530635]
1959 1962 [5.56546066 7.34032487 9.46552441 9.81585975]
1962 1965 [5.32462144 5.69774766 6.53469084 6.99850779]
1965 1968 [5.60732579 6.58548165 7.4648404 ]
1968 1971 [ 5.35872342  6.96786087  9.96588369 11.35476906]
1971 1974 [5.31423897 5.42072168 5.77672445 6.02624452]
1974 1977 [5.383831   6.33756708 7.66087795]
1977 1980 [5.30508773 5.5017374  6.3057585  6.91312993]
1980 1983 [5.44800101 7.79144787 9.93418401]
1983 1986 [5.39816833 5.87582415 6.92156355 7.48964713]
1986 1989 [5.29738954 7.26253828 9.15981429]
1989 1992 [5.36578603 5.38417632 6.68022905 7.95789

...

22668 22671 [ 6.62961284  8.34348413 10.55734354]
22671 22674 [5.30270798 5.30948749 5.50155716 5.68684732]
22674 22677 [5.42296527 7.18922388 8.95438919]
22677 22680 [5.53555631 5.56483198 6.55137809 7.50864851]
22680 22683 [5.36230354 5.66988798 6.25855624 6.53964006]
22683 22686 [5.20616343 7.3735414  9.42162846]
22686 22689 [5.27075276 6.47788144 7.52178712]
22689 22692 [5.32366063 5.36477626 5.43128295 5.45667401]
22692 22695 [5.43632153 5.85578663 6.36563971 6.45602768]
22695 22698 [ 6.70042864  6.96119709  9.62078655 12.01960755]
22698 22701 [5.25012802 5.25353195 5.3595483  5.46216073]
22701 22704 [5.45048321 5.52210313 5.92929907 6.26487509]
22704 22707 [5.29509767 7.58126679 9.35161025]
22707 22710 [5.42769379 6.81058954 8.95941974 9.7253542 ]
22710 22713 [7.53277749 7.76031526 8.02963724 8.07142146]
22713 22716 [5.47465668 6.43298583 7.87543352 8.35955206]
22716 22719 [5.5428111  7.06045839 9.05569199 9.53327829]
22719 22722 [5.29776571 5.41252061 5.58120192 5.63512833]
2272

23124 23127 [5.36480857 5.44615543 5.58890555 5.65030881]
23127 23130 [5.69796544 6.09267938 6.4174492 ]
23130 23133 [5.36758155 5.49624891 6.56329439 7.50167253]
23133 23136 [5.16177845 5.64661713 7.30357194 8.47568806]
23136 23139 [5.24233531 5.31187928 5.38560376]
23139 23142 [5.46013723 5.64563619 6.8154005  7.79966585]
23142 23145 [5.13076603 5.21055428 6.09331918 6.89629583]
23145 23148 [ 5.1994197   8.72810037 11.92450751]
23148 23151 [5.88984051 5.97990602 6.31207169 6.55417184]
23151 23154 [ 7.13316139  7.34638726  8.85555948 10.15150582]
23154 23157 [5.5373578  5.5502111  5.87932363 6.19558285]
23157 23160 [5.30752873 5.57176662 6.04461485 6.25322519]
23160 23163 [5.37168401 5.40400291 5.47414087 5.51195993]
23163 23166 [5.35475489 6.01323695 7.09526393 7.51880886]
23166 23169 [5.30174663 5.40556094 6.59112794 7.67288063]
23169 23172 [ 5.18049095  5.42984027  8.44588652 11.21258345]
23172 23175 [6.12186868 6.79779999 7.60634216]
23175 23178 [5.65334278 6.28976806 6.77437557]


23571 23574 [5.33318159 5.44524704 5.56458972 5.57186696]
23574 23577 [5.33636619 5.42383762 6.20628812 6.9012672 ]
23577 23580 [5.25194589 5.70513189 6.13228216]
23580 23583 [ 5.32535714  7.43712497  9.95299696 10.35710113]
23583 23586 [5.16677354 5.54681894 7.80608621 9.68530808]
23586 23589 [5.17068445 5.37960632 5.83380514 6.07908208]
23589 23592 [5.26892425 5.39228442 6.27079788 7.02595116]
23592 23595 [5.79484415 6.25337649 8.03885647 9.3658041 ]
23595 23598 [5.15336135 6.56899647 7.88380033]
23598 23601 [5.33375708 5.4427432  5.66599668 5.78026404]
23601 23604 [ 5.46634146  8.50831156 11.49773542]
23604 23607 [ 5.62891978  9.61326705 12.90895186]
23607 23610 [5.49798291 5.66725853 7.10865874 8.38078333]
23610 23613 [5.23333554 5.40173984 6.03151707 6.49289002]
23613 23616 [5.31943281 5.44626066 6.81905521 8.06502191]
23616 23619 [ 5.1854006   5.3217405   7.83715861 10.21623682]
23619 23622 [5.74317465 6.14525331 6.95906857 7.37080518]
23622 23625 [5.79671198 6.76813481 8.0422230

24024 24027 [5.37796944 6.62579654 7.75252033]
24027 24030 [5.39523083 5.76899609 6.8850156  7.62726985]
24030 24033 [5.29124527 5.42473834 5.75765139 5.95707137]
24033 24036 [5.77367074 6.64124838 7.80146983]
24036 24039 [5.43811377 6.64478701 7.80028242]
24039 24042 [5.21992472 5.27302843 5.48258757 5.639043  ]
24042 24045 [5.2585964  6.92370297 8.33003851]
24045 24048 [5.30722724 6.2287268  6.98964105]
24048 24051 [5.45414829 5.56444038 5.82835753 5.98198258]
24051 24054 [5.41704128 5.49318524 5.64505726 5.7207853 ]
24054 24057 [5.42211104 6.28161133 7.00934791]
24057 24060 [5.44340068 5.45871334 5.58538309 5.69674018]
24060 24063 [5.47471175 5.5895627  5.70238252]
24063 24066 [5.23447924 5.27824005 5.40904666 5.49609247]
24066 24069 [5.42410552 5.49175159 6.58206654 7.60473542]
24069 24072 [5.21463169 6.56338035 7.69781808]
24072 24075 [5.3083737  6.35079155 7.13409419]
24075 24078 [5.27759946 6.20962102 7.29875437]
24078 24081 [6.87894556 7.67919652 8.72538605 8.97132462]
24081 24

ValueError: n_samples=2 should be >= n_clusters=3

In [192]:
X_binned.to_csv(r'X_binned.csv', index = True)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [191]:
X_binned.shape

(24366, 1904)

In [137]:
X_binned = enc.fit_transform(X)
enc.bin_edges_[0] #these are the discrete bins
    

array([ 4.71933061,  6.57409762,  8.79721917, 14.45875141])

array([[1., 2., 2., ..., 2., 2., 1.],
       [0., 1., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [193]:
ss=np.array(X_binned)
ss

array([[1., 2., 1., ..., 2., 2., 2.],
       [0., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [2., 1., 2., ..., 1., 2., 1.]])

In [93]:
for x in range(24368):
    for y in range(1904):
        if ss[x,y] == 0: #this is high end
            ss[x,y] = -2
        elif ss[x,y] == 1:  #this is the average value 
            ss[x,y] = -1
        elif ss[x,y] == 2:  #this is the average value 
            ss[x,y] = 0
        elif ss[x,y] == 3:  #this is the average value 
            ss[x,y] = 1
        elif ss[x,y] == 4:  #this is the average value 
            ss[x,y] = 2
       

In [198]:
sa=pd.DataFrame(ss,index=genes[0:24366],columns=patients)

In [197]:
genes[0:24366]

array(['DA110839', 'FAM71A'], dtype=object)

In [200]:
sa=sa.T

In [222]:
sa

,RERE,RNF165,CD049690,BC033982,PHF7,CIDEA,PAPD4,AI082173,SLC17A3,SDS,...,LOC283104,VN1R4,BX115874,BX107598,UGCGL1,VPS72,CSMD3,CC2D1A,CB986545,IGSF9
MB-0362,1.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,1.0,2.0,...,0.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,2.0
MB-0346,2.0,1.0,0.0,1.0,2.0,0.0,2.0,0.0,1.0,1.0,...,1.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
MB-0386,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,1.0,...,0.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0
MB-0574,2.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,2.0,...,0.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,2.0
MB-0503,2.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,2.0,...,1.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,1.0,2.0,0.0,0.0,2.0,1.0,2.0,0.0,1.0,1.0,...,0.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,0.0,2.0
MB-5453,2.0,1.0,0.0,0.0,2.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0
MB-5471,2.0,1.0,0.0,0.0,1.0,2.0,2.0,0.0,1.0,2.0,...,0.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
MB-5127,2.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,2.0,...,0.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0


In [21]:
#normalising code in case needed 
x = ds.values
min_max_scaler = preprocessing.MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(x)
x_scaled

array([[0.40635351, 0.50373666, 0.43574454, ..., 0.44538647, 0.42513951,
        0.37973505],
       [0.13922802, 0.2013728 , 0.12075052, ..., 0.21949703, 0.18843652,
        0.21728095],
       [0.07542513, 0.0755925 , 0.07946281, ..., 0.06433787, 0.08748843,
        0.08495792],
       ...,
       [0.26544823, 0.32834954, 0.28915178, ..., 0.29614064, 0.19563092,
        0.27936406],
       [0.05467595, 0.07522815, 0.05261233, ..., 0.07109958, 0.09421473,
        0.08999519],
       [0.03390969, 0.06158464, 0.07543188, ..., 0.03917326, 0.06540295,
        0.06319818]])

In [22]:
saa=pd.DataFrame(x_scaled,index=genes,columns=patients)

In [23]:
saa=saa.T
saa

,RERE,RNF165,CD049690,BC033982,PHF7,CIDEA,PAPD4,AI082173,SLC17A3,SDS,...,BX115874,BX107598,UGCGL1,VPS72,CSMD3,CC2D1A,CB986545,IGSF9,DA110839,FAM71A
MB-0362,0.406354,0.139228,0.075425,0.028256,0.114888,0.172307,0.327215,0.055458,0.100935,0.227058,...,0.056678,0.098670,0.304860,0.345570,0.045430,0.167760,0.012029,0.265448,0.054676,0.033910
MB-0346,0.503737,0.201373,0.075592,0.064564,0.090548,0.054400,0.362395,0.068036,0.082485,0.145686,...,0.125417,0.076294,0.315181,0.370741,0.049412,0.144734,0.061588,0.328350,0.075228,0.061585
MB-0386,0.435745,0.120751,0.079463,0.053830,0.132838,0.544508,0.327378,0.055299,0.098422,0.183133,...,0.046470,0.059536,0.324858,0.372472,0.340333,0.166693,0.075916,0.289152,0.052612,0.075432
MB-0574,0.422136,0.093952,0.077801,0.061792,0.116684,0.144231,0.462465,0.070038,0.074421,0.176496,...,0.041214,0.094313,0.311891,0.370136,0.109539,0.175872,0.049084,0.225274,0.038657,0.052584
MB-0503,0.467647,0.122559,0.083908,0.054414,0.128297,0.319372,0.409446,0.046531,0.072193,0.202146,...,0.070637,0.077801,0.306373,0.351039,0.080467,0.153894,0.057542,0.217990,0.061708,0.032203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,0.350587,0.450092,0.072819,0.023212,0.095540,0.092115,0.316400,0.051995,0.090735,0.191828,...,0.072255,0.079143,0.300573,0.341192,0.076289,0.159651,0.064077,0.264105,0.066276,0.055454
MB-5453,0.501877,0.278377,0.084211,0.035815,0.124544,0.124505,0.390470,0.042718,0.085897,0.115731,...,0.087514,0.084665,0.239555,0.291976,0.070151,0.243632,0.051639,0.260005,0.085253,0.074192
MB-5471,0.445386,0.219497,0.064338,0.046612,0.144220,0.170638,0.388931,0.025507,0.080587,0.168289,...,0.079214,0.069154,0.310232,0.331645,0.072204,0.143541,0.073399,0.296141,0.071100,0.039173
MB-5127,0.425140,0.188437,0.087488,0.043809,0.125463,0.128203,0.472475,0.071380,0.100228,0.263256,...,0.070773,0.074209,0.266510,0.323406,0.054304,0.164425,0.027127,0.195631,0.094215,0.065403


In [24]:
saa.to_csv(r'gene_nor.csv', index = True)

In [202]:
#saved this to a file for mrmr
sa.to_csv(r'gene_dis6hr.csv', index = True)
#ends here

In [207]:
dq = pd.read_csv ('brca_metabric/data_CNA.csv')# will import datset



In [208]:
dq=dq.T
dq=dq.reset_index()
dq

,index,0,1,2,3,4,5,6,7,8,...,22534,22535,22536,22537,22538,22539,22540,22541,22542,22543
0,Hugo_Symbol,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
1,MB-0000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MB-0039,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-1,0,0
3,MB-0045,-1,-1,0,-1,-1,-1,-1,0,0,...,0,0,0,0,0,1,-1,0,-2,0
4,MB-0046,0,0,0,-1,-1,-1,-1,0,-1,...,0,0,-1,-1,0,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2169,MB-6020,1,1,0,0,0,0,0,0,-1,...,0,0,-1,-1,0,0,0,0,-1,2
2170,MB-6213,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2171,MB-6230,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-1,-1,0,0,0
2172,MB-7148,0,0,-1,0,0,0,0,-1,0,...,0,-1,0,0,0,0,0,0,0,0


In [209]:
geness=np.array(dq.iloc[0,1:])
geness

array(['A1BG', 'A1BG-AS1', 'A1CF', ..., 'ZYX', 'ZZEF1', 'ZZZ3'],
      dtype=object)

In [210]:
patientss=np.array(dq.iloc[1:,0])
patientss

array(['MB-0000', 'MB-0039', 'MB-0045', ..., 'MB-6230', 'MB-7148',
       'MB-7188'], dtype=object)

In [211]:
dp=np.array(dq.iloc[1:,1:])

In [212]:
cnn=pd.DataFrame(dp,index=patientss,columns=geness)

In [123]:
#saved this to a file for mrmr
cnn.to_csv(r'CNA_ffR.csv', index = True)
#ends here

In [214]:
#get CNA data
excna= pd.read_csv ('brca_metabric/gene_datas.csv',encoding="latin-1'")# will import datset
excna=excna.drop(columns=['OrderÊ', 'FeaÊ'])
excna = excna.replace({'Ê':''}, regex=True)
excna

,NameÊ,Score
0,DDX11,0.000
1,KMO,-0.004
2,LINC00229,-0.008
3,DBF4,-0.012
4,SSSCA1,-0.014
...,...,...
195,LINC00659,-0.045
196,LINC00299,-0.045
197,SULT4A1,-0.045
198,LINC00305,-0.045


In [215]:
#transpose for using 
cna=cnn.T


In [216]:
#remove index for join 
cna=cna.reset_index()

In [217]:
#join
join_cna=excna.set_index('NameÊ').join(cna.set_index('index'))
join_cna=join_cna.drop(columns=['Score'])
join_cna
join_cna

,MB-0000,MB-0039,MB-0045,MB-0046,MB-0048,MB-0050,MB-0053,MB-0062,MB-0064,MB-0066,...,MB-5467,MB-5546,MB-5585,MB-5625,MB-5648,MB-6020,MB-6213,MB-6230,MB-7148,MB-7188
NameÊ,,,,,,,,,,,,,,,,,,,,,
DDX11,0,1,0,-1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
KMO,0,0,0,1,0,2,0,2,1,0,...,0,0,1,-1,1,0,0,1,1,0
LINC00229,0,0,0,-1,-1,-1,0,1,0,-1,...,0,0,0,0,-1,-1,0,0,0,0
DBF4,0,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,-1
SSSCA1,0,-1,0,0,1,0,0,0,0,0,...,0,0,-1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LINC00659,0,0,0,0,0,0,2,-1,0,1,...,1,0,0,0,0,1,0,0,0,1
LINC00299,0,0,0,0,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0
SULT4A1,0,0,0,-1,-1,-1,0,1,0,-1,...,0,0,0,0,-1,-1,0,0,0,0


In [220]:
join_cna=join_cna.T
join_cna

NameÊ,DDX11,KMO,LINC00229,DBF4,SSSCA1,DBR1,CYP7A1,CYBA,PGLS,CTNNA2,...,LINC00358,SPDYE8P,SUCLG2,PGA5,LINC00515,LINC00659,LINC00299,SULT4A1,LINC00305,PCSK4
MB-0000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MB-0039,1,0,0,-1,-1,0,0,0,0,0,...,-1,0,0,-1,0,0,0,0,0,0
MB-0045,0,0,0,0,0,2,0,0,0,0,...,0,-1,0,0,0,0,0,0,-1,0
MB-0046,-1,1,-1,0,0,0,0,-1,-1,0,...,-1,-1,0,0,0,0,0,-1,-1,-1
MB-0048,0,0,-1,0,1,0,1,-1,2,1,...,0,0,0,0,0,0,0,-1,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-6020,0,0,-1,0,0,0,0,-1,0,0,...,0,0,0,0,1,1,0,-1,0,0
MB-6213,0,0,0,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
MB-6230,0,1,0,0,0,0,0,-1,0,0,...,0,-2,0,0,0,0,0,0,0,0
MB-7148,0,1,0,0,0,0,0,-1,0,0,...,0,0,-1,0,0,0,0,0,0,0


In [221]:
join_cna.to_csv(r'after_mrmr_CNA.csv', index = True)

In [223]:
#get gene data
exexp= pd.read_csv ('brca_metabric/expex.csv',encoding="latin-1'")# will import datset
exexp=exexp.drop(columns=['OrderÊ', 'FeaÊ'])
exexp = exexp.replace({'Ê':''}, regex=True)
exexp

,NameÊ,Score
0,AA884745,0
1,GLYCTK,0
2,PLEKHG1,0
3,AA620691,0
4,DAB1,0
...,...,...
395,BF115640,0
396,CAMK2B,0
397,FAM70A,0
398,WHSC1,0


In [225]:
sa.T

,MB-0362,MB-0346,MB-0386,MB-0574,MB-0503,MB-0641,MB-0201,MB-0218,MB-0316,MB-0189,...,MB-6122,MB-6192,MB-4820,MB-5527,MB-5167,MB-5465,MB-5453,MB-5471,MB-5127,MB-4313
RERE,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0
RNF165,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0
CD049690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BC033982,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PHF7,1.0,2.0,0.0,1.0,0.0,0.0,2.0,2.0,2.0,0.0,...,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VPS72,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0
CSMD3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC2D1A,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
CB986545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [227]:
#join
join_g=exexp.set_index('NameÊ').join(sa.T)
join_g=join_g.drop(columns=['Score'])
join_g

,MB-0362,MB-0346,MB-0386,MB-0574,MB-0503,MB-0641,MB-0201,MB-0218,MB-0316,MB-0189,...,MB-6122,MB-6192,MB-4820,MB-5527,MB-5167,MB-5465,MB-5453,MB-5471,MB-5127,MB-4313
NameÊ,,,,,,,,,,,,,,,,,,,,,
AA884745,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
GLYCTK,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,...,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0
PLEKHG1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
AA620691,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
DAB1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BF115640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CAMK2B,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,0.0,0.0,...,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0
FAM70A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
join_g.to_csv(r'after_mrmr_geneexp.csv', index = True)